In [ ]:
%pip install crewai==0.28.8 crewai_tools==0.1.6 langchain-groq sentence-transformers langchain_huggingface langchain-openai langchain-community==0.0.29


In [ ]:
# 📥 Import necessary packages and modules
from langchain_openai import ChatOpenAI
import os
from crewai_tools import PDFSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools import tool
from crewai import Crew
from crewai import Task
from crewai import Agent

In [ ]:
# 🔐 Load API keys
GROQ_API_KEY='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'

In [ ]:
# 🧠 Set up the LLM model using Groq's Llama3
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile",
    temperature=0.1,
    max_tokens=1000,
)

In [ ]:
rag_tool = PDFSearchTool(pdf='doc.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama-3.3-70b-versatile",
                api_key=GROQ_API_KEY,
                # max_tokens=1000,
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)


In [ ]:
# 🚀 Query the RAG tool to perform document-based search
rag_tool.run("What does Sporo Health do?")

Using Tool: Search a PDF's content


"Relevant Content:\nincreasing capabilities and token storage of LLMs in 2024 such as Google’s Med-Gemini, Meta’s Llama 3, OpenAI’s ChatGPT4, or Anthropic’s Claude 3.5, has allowed for these models to process the enormous portions of information for summarization and analysis. In recognition of the stringent accuracy, the need for personalization, privacy regulations, and the high knowledge floor needed for AI in clinical workflow, the innovation space gave birth to companies like Sporo Health to combat the aforementioned issues in clinical summarization using AI agents. Several case studies verify AI usage in various clinical settings to aid in chart review and summarization of clinical information. 3\n\nASSESSING THE ROLE OF CLINICAL SUMMARIZATION AND PATIENT CHART REVIEW WITHIN COMMUNICATIONS , MEDICAL MANAGEMENT ,AND DIAGNOSTICS Chanseo Lee Yale School of Medicine and Sporo Health New Haven, CT 06510 chanseo.lee@yale.eduKimon-Aristotelis Vogt Sporo Health Boston, MA 02134 kvogt@spo

In [ ]:
import os
TAVILY_API_KEY='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'
# Set Tavily API key as an environment variable
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY
# 🌐 Set up Tavily tool for real-time web search
web_search_tool = TavilySearchResults(k=3)

In [ ]:
#Run a web search using the Tavily tool
web_search_tool.run("What does Sporo Health do?")

[{'url': 'https://app.dealroom.co/companies/sporo_health',
  'content': "Sporo Health is a pioneering MedTech startup focused on revolutionizing healthcare through advanced generative AI solutions. The company operates in the healthcare technology market, specifically targeting clinicians and healthcare providers. Sporo Health's core product is an AI-driven platform that automates clinical workflows, allowing clinicians to focus more on personalized patient care rather than administrative tasks. By analyzing comprehensive patient data, the AI enables the creation of [...] personalized care plans and interactions, improving treatment outcomes and patient satisfaction. The platform also ensures data privacy and compliance through its advanced de-identification framework, safeguarding sensitive patient information. Sporo Health's business model involves partnering with healthcare institutions and leveraging strategic alliances, such as its partnership with Microsoft and participation in t

In [ ]:
@tool
def router_tool(keywords):
  """Router Function"""


  if 'Sporo Health' in keywords:
    return 'vectorstore'
  else:
    return 'web_search'

In [ ]:
# 🤖 Define an AI agent with its role and toolset
Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [ ]:
# 🤖 Define an AI agent with its role and toolset
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [ ]:
# 🤖 Define an AI agent with its role and toolset
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [ ]:
# 🤖 Define an AI agent with its role and toolset
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [ ]:
# 🤖 Define an AI agent with its role and toolset
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [ ]:
#Student task - new AI agent with new role

evaluator = Agent(
    role='Evaluator',
    goal='Evaluate the performance of the other agents and refine the task',
    backstory=(
        "You are an evaluator assessing the performance of the other AI agents and refining the task"
        "You evaluate the output of the other AI agents and decide whether they have adequately performed their task"
        "If the agents have performed their task well, proceed with the generated answer"
        "If the agents have not performed their task well, generate a new anwer"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
    )



In [ ]:
# 🧩 Define the task each agent is responsible for
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Use only a list of key words as as input for 'router_tool"
    "If the router_tool returns vectorstore, return a single word 'vectorstore' if it is eligible for vectorstore search."
    "If the router_tool returns websearch, return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [ ]:
# 🧩 Define the task each agent is responsible for
retriever_task = Task(
    description=(
    "Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a clear and concise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
    #tools=[retriever_tool],
)

In [ ]:
# 🧩 Define the task each agent is responsible for
grader_task = Task(
    description=("Based on the response from the retriever task for the question {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [ ]:
# 🧩 Define the task each agent is responsible for
hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

# 🧩 Define the task each agent is responsible for
answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)

In [ ]:
#Define evaluator task
evaluator_task = Task(
    description =("Based on the outputs from prior tasks, evaluate the degree to which each agent has performed it's role"),
    expected_output=("Binary decision indicating whether the agents have performed their tasks adequately resulting in an acceptable answer"
    "If decision is yes, respond with ONLY the answer generated by the 'answer_task'"
    "There is no need to provide an explanation if the agents have performed well. Only return the final answer generated by 'answer_task'"
    "Otherwise respond 'no' and suggest concise areas for improvement"),
    agent=evaluator,
    context=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    )

In [ ]:
# 👥 Assemble agents into a Crew to collaborate on the task
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader, evaluator],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task, evaluator_task],
    verbose=True,
)

In [ ]:
inputs ={"question":"Does Sporo Health Streamline patient chart reviews?"}

In [ ]:
# ▶️ Start the collaborative process among the agents
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Does Sporo Health Streamline patient chart reviews?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Use only a list of key words as as input for 'router_toolIf the router_tool returns vectorstore, return a single word 'vectorstore' if it is eligible for vectorstore search.If the router_tool returns websearch, return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01ka770vqfek3bfr93a214h186` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99924, Requested 437. Please try again in 5m11.904s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
print(result)

In [ ]:
rag_tool.add("/content/2410.15528v1.pdf")
#Add more contents to rag_tools's vectorstore

In [ ]:
rag_tool.run("What is the focus of Sporo health?")

#Rag_tool.run acts as a similarity_search in this case. However for the purpose of learning I will also define
#a vectorstore below and explore this

In [ ]:
#Prepare documents for vectorstore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

#Load PDF and split into chunks
loader = PyPDFLoader("/content/2410.15528v1.pdf")
doc = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(doc)

#Repeat for other pdf
loader_1 = PyPDFLoader("/content/doc.pdf")
doc_1 = loader_1.load_and_split()
text_splitter_1 = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks_1 = text_splitter_1.split_documents(doc_1)


In [ ]:
#Import FAISS
%pip install faiss-cpu
from langchain.vectorstores import FAISS

In [ ]:
#Initialise vectorstore

#Use HuggingFace

embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

vectorstore = FAISS.from_documents(chunks, embedding_model)


In [ ]:
#Perform similarity search

#Search query

query = "What is the focus of Sporo health?"

#Search for top 3 most relevant documents

retrieved_docs = vectorstore.similarity_search(query, k=3)

for doc in retrieved_docs:
    # Print the first 300 characters of each document chunk
    print(doc.page_content[:300], "...\n")

In [ ]:
from sklearn.decomposition import PCA
#
#Perform PCA and visualise embeddings

#Generate embeddings
embeddings = embedding_model.embed_documents([doc.page_content for doc in chunks])
#Perform PCA

pca = PCA(n_components=2)
pca_result = pca.fit_transform(embeddings)

print (pca_result)

In [ ]:
#Visualise

import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.scatter(pca_result[:, 0], pca_result[:, 1])
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
plt.show()

In [ ]:
#Add new knowledge to knowledge base
#Already proessed second PDF above

vectorstore.add_documents(chunks_1)

In [ ]:
#Test model with new queries

query = "What is the focus of Sporo health?"

#Search for top 3 most relevant documents

retrieved_docs = vectorstore.similarity_search(query, k=3)
for doc in retrieved_docs:
    print(doc.page_content[:300])

#Adding more to knowledge base has changed the result

In [ ]:
#Create new AI agent and add to crew - above


In [ ]:
#Set up capability for reporter to generate PHDs
%pip install reportLab


In [ ]:
import reportlab

In [ ]:
#Define tool for reporter agent to use

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
@tool
def pdf_exporter(text):
  """PDF Exporter"""
  doc = SimpleDocTemplate("output.pdf", pagesize=letter)
  styles = getSampleStyleSheet()
  elements = []
  for line in text.split("\n"):
    elements.append(Paragraph(line, styles["Normal"]))
    elements.append(Spacer(1, 12))
  doc.build(elements)
  return "output.pdf"



In [ ]:
#Report generator agent
#Define the agent's job
reporter = Agent(
    role='Findings reporter',
    goal='Collate the findings from a query into an easily digestible report',
    backstory=(
        "You are a reporter, master at summarising complex information into an easily digestible form"
        "You take results returned from vectorstores or web searches and collate them into easy-to-read documents"
        "You work with retrieval and checking agents who will provide you with a checked and cleaned set of information to summarise"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
    )
reporter_task = Task(
    description = ("Collate the information returned by vectorstore or web searches into an easily readable format. "
                   "Use the 'pdf_exporter' tool to return your findings as a PDF."
                   "Use the label 'text' as the input for the 'pdf_reporter' tool"),
    expected_output=("A PDF document summarising the key points of the vectorstore or web searches"),
    agent=reporter,
    context=[answer_task],
    tools=[pdf_exporter]
    )

In [ ]:
#Adjust other agents - if the reporter is going to summarise finding the retriever can be more liberable with the information they provide
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a detailed answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a clear and detailed as response."),
    agent=Retriever_Agent,
    context=[router_task],
   #tools=[retriever_tool],
)

In [ ]:
#Define a new answer grading agent specific to this task
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant, pass the response to the reporter to be summarised."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool' and pass this to the reporter"
        "There is no need to expound if the answer is relevant, simply pass it along"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)
answer_task = Task(
    description = ("Evaluate the answer provided by the retriever task and decide whether it is relevant to answer the question"
    "If the answer is relevant, pass it along to the reporter"
    "If the answer is not relevant, perform a web search using 'web_search_tool' and pass this along instead"),
    agent = answer_grader,
    context=[retriever_task],
    expected_output=("If the answer provided by the retriever is relevant, output the entire answer"
    "If the answer is not relevant, return a web search"),
)



In [ ]:
#Assemble new crew

rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, answer_grader, reporter],
    tasks=[router_task, retriever_task, answer_task, reporter_task],
    verbose=True,
)

In [ ]:
#Query the crew

inputs ={"question":"Does Sporo Streamline patient chart reviews?"}
result = rag_crew.kickoff(inputs=inputs)

In [ ]:
print (result)